<a href="https://colab.research.google.com/github/EKCA040602/Geoestadistica-/blob/main/GEOESTADISTICA_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


dataMadre=pd.read_excel("/content/DATA_MADRE.xlsx",header=1)


In [ ]:
#Limpieza de data

Nombre_de_columna=list(dataMadre.columns)

def primera_limpieza():
  columna_vacia=[]
  for i in Nombre_de_columna:
    if  dataMadre[i].dtype=="object" and all (dataMadre[i].isin(["N.R."])):
        columna_vacia.append(i)
  df=dataMadre.drop(columna_vacia,axis=1)
  return df

tabla_de_limpieza_de_NR=primera_limpieza()



In [ ]:
Nombre_de_columna_filtradas=list(tabla_de_limpieza_de_NR.columns)
Nombre_de_columna_filtradas
# len(Nombre_de_columna_filtradas)

In [ ]:
tabla_de_verificacion=pd.read_excel("/content/VERIFICACION_DE_ELEMENTOS.xlsx")
# p=tabla_de_verificacion[tabla_de_verificacion["SIMBOLO"]=="U"]

# q=tabla_de_verificacion[tabla_de_verificacion["SIMBOLO"].str.contains("a")]
# print(q)
# metodo=tabla_de_verificacion[ tabla_de_verificacion[tabla_de_verificacion["1ER METODO"]=="4 acidos"] and tabla_de_verificacion[tabla_de_verificacion["2DO METODO"]=="4 acidos" ]]

#Codifique con  que metodo esta trabajando
def metodo(met):
  metodo=tabla_de_verificacion[(tabla_de_verificacion["1ER METODO"]==met) | (tabla_de_verificacion["2DO METODO"]==met)]
  elementos=list(metodo["SIMBOLO"])
  return elementos

elementos_validos=metodo("4 acidos")


In [ ]:
# tabl_de_limpieza_de_NR.shape
len(elementos_validos)

49

In [ ]:
# Elementos_de_una_letra=["H","B","C","N","O","F","P","S","K","I","Y","V","W"]

def filtro():
  new_filtro=[]
  for i in  Nombre_de_columna_filtradas:
    #CHEQUEAR EL ORO
    if i.startswith("Au"):
      new_filtro.append(i)

    for j in elementos_validos :
      # if i.startswith(j) and (i.endswith("ppb") or i.endswith("ppm") or i.endswith("pct")):
      if (i.startswith(f"{j}_") and len(i)==5) or  (i.startswith(f"{j}_") and len(i)==6) or (i.startswith(f"{j}_1")):
        new_filtro.append(i)
  return new_filtro

elementos_aprobados=filtro()
elementos_aprobados
# len(elementos_aprobados)

In [ ]:
cleanTableFor4acid=tabla_de_limpieza_de_NR[elementos_aprobados]
cleanTableFor4acid

#Falta parte de censura

In [ ]:
for i in elementos_aprobados:
  cleanTableFor4acid[i] = cleanTableFor4acid[i].astype(object)
  cleanTableFor4acid[i] .dtype

In [ ]:
cleanTableFor4acid

In [ ]:
longitud=len(cleanTableFor4acid)
longitud

829

In [ ]:
def censura_elemento():
  lc=[]
  for i  in elementos_aprobados:
    for j in range(0,longitud):
      if (str(cleanTableFor4acid[i][j]).startswith("<") or str(cleanTableFor4acid[i][j]).startswith(">")):
        lc.append(i)
  lc=list(set(lc))
  return lc

elementos_a_chequear=censura_elemento()

elementos_a_chequear

# def modificar(datos):
#   cleanTableFor4acid[datos]




In [ ]:
def censura_elemento_porcentaje():
  porcentaje=[]
  for i  in elementos_a_chequear:
    c=0
    for j in range(0,longitud):
      if (str(cleanTableFor4acid[i][j]).startswith("<")):
        c+=1
    d=round(((c*100)/longitud),2)
    porcentaje.append(d)
  return porcentaje

prcj=censura_elemento_porcentaje()
columnas_censuradas=dict(zip(elementos_a_chequear,prcj))
columnas_censuradas

In [ ]:
prcj=censura_elemento_porcentaje()
columnas_censuradas=dict(zip(elementos_a_chequear,prcj))
#Aqui esta las columnas que deben ser censuradas
c_c=[]
for i,j in columnas_censuradas.items():
  if columnas_censuradas[i]>=10 :
    c_c.append(i)
#SOLO PARA ORO
c_c.remove('Au_ppb')
c_c

['In_ppm', 'Te_ppm', 'Na_pct', 'Se_ppm', 'Hf_ppm']

In [ ]:
cleanTableFor4acid[cleanTableFor4acid["Au_ppb"]=="<5"]

In [ ]:
def censurados():
  for i  in elementos_aprobados:
    for j in range(0,longitud):
      if str(cleanTableFor4acid[i][j]).startswith("<") :
          cleanTableFor4acid[i][j]=str(float(cleanTableFor4acid[i][j][1:])/2)
      if str(cleanTableFor4acid[i][j]).startswith(">"):
          cleanTableFor4acid[i][j]=str(float(cleanTableFor4acid[i][j][1:])+0.1)
  return cleanTableFor4acid

data_casi_limpia=censurados()
data_limpia_elementos=data_casi_limpia.drop(c_c,axis=1)

#DATA PARA ANALIZAR


columnas_limpias=list(data_limpia_elementos.columns)

#De STRING A FLOAT

for i in columnas_limpias:
  data_limpia_elementos[i] = data_limpia_elementos[i].astype(float)
  data_limpia_elementos[i] .dtype

data_limpia_elementos.to_excel("/content/TABLA_DE_SOLO_ELEMENTOS.xlsx",sheet_name="pag",)

In [ ]:
#DATA CON DESCRIPCION Y ELEMENTOS


Nombre_de_columna=Nombre_de_columna[:25]
data_limpia_desc_y_ele=dataMadre[Nombre_de_columna].copy()
data_limpia_desc_y_ele[columnas_limpias]=data_limpia_elementos[columnas_limpias]

data_limpia_desc_y_ele.to_excel("/content/TABLA_DE_ELEMENTOS_Y_DESCRIPCION.xlsx",sheet_name="pag",)

In [ ]:
#GRAFICOS HISTOGRAMA Y BOXPLOT
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


elemnt_con_log=pd.read_excel("/content/TABLA_DE_SOLO_ELEMENTOS_CON_ORO_LOG.xlsx")
elemnt_con_log.describe()

Si la prueba de Shapiro-Wilk produce un valor p alto (por ejemplo, **superior a 0.05**), el **histograma muestra una forma de campana y los puntos en el QQ plot se encuentran cerca de una línea recta,** entonces es probable que los datos sigan una distribución normal.
Si la prueba de Shapiro-Wilk produce un valor p bajo (por ejemplo, inferior a 0.
05), el histograma tiene una forma diferente a la campana o los puntos en el QQ plot se desvían significativamente de la línea recta, entonces es menos probable que los datos sigan una distribución normal.

In [ ]:
#BUSCA EN EL LINK CUANDO LLEGUES A QQPLOT
#https://seaborn-qqplot.readthedocs.io/en/latest/


In [ ]:
# Statsmodels: Ofrece la función qqplot para generar gráficos QQ.---OJITO

# import plotly.graph_objects as go

# import ipywidgets as widgets

# def graficos_plotly(header):
#   trace1=go.Box(y=elemnt_con_log[header],name=header,boxpoints='suspectedoutliers', marker=dict(
#         color='rgb(8,81,156)',
#         outliercolor='rgba(219, 64, 82, 0.6)',
#         line=dict(
#             outliercolor='rgba(219, 64, 82, 0.6)',
#             outlierwidth=2)),
#     line_color='rgb(8,81,156)')
#   trace2=go.Histogram(x=elemnt_con_log[header],name=header,nbinsx=30)
#   fig = go.Figure(data=[ trace2])
#   fig.show()

# graficos_plotly("S_pct LOG")
elemnt_con_log.describe().T

Chamba de 8.10-->Retiro de valores atipicos[circulo] y atipicos extremos[estrella][GENERA QUE POBLACION NO SE PUEDA NORMALIZAR]

Identificamos con la formula estos valores atipicos y atipicos extremos (limite donde inicia el atipico) y eliminamos esa data del excel que tengan valoress mayores a ese numero.


In [ ]:
#VAMOS A HACER UN FILTRADO Y N VECES PARA TENER BUENA DATA
def filtrado_de_outlayers(numero):
  for i in range(0,numero):
    calculo_cuartil=elemnt_con_log.describe().T
#CALCULO DE BIGOTE INFERIOR Y SUPERIOR
    inferior=list(calculo_cuartil["25%"]-1.5*(calculo_cuartil["75%"]-calculo_cuartil["25%"]))
    superior=list(calculo_cuartil["75%"]+1.5*(calculo_cuartil["75%"]-calculo_cuartil["25%"]))
    calculo_cuartil["BIGOTE_INFERIOR"]=inferior
    calculo_cuartil["BIGOTE_SUPERIOR"]=superior
    elementos=list(elemnt_con_log.columns)
    fila_cantidad=len(elemnt_con_log)
    for  i,infe,supe in zip(elementos,inferior,superior):
      for j in range(0,fila_cantidad):
        if float(elemnt_con_log[i][j])<infe or float(elemnt_con_log[i][j])>supe:
          elemnt_con_log[i][j]=None

  return elemnt_con_log.describe().T.to_excel("/content/TABLA_DE_DATOS_ESTADISTICOS.xlsx",sheet_name="Hoja1",)

filtrado_de_outlayers(4)

In [ ]:
# elemnt_con_log.describe().T

Nombre_de_columna=Nombre_de_columna[:25]
log_conBS_Y_BI=dataMadre[Nombre_de_columna].copy()

log_conBS_Y_BI[list(elemnt_con_log.columns)]=elemnt_con_log[list(elemnt_con_log.columns)]
log_conBS_Y_BI

log_conBS_Y_BI.to_excel("/content/log_conBS_Y_BI.xlsx",sheet_name="Hoja1",)

In [ ]:
#Capping de 95%

ele=list(elemnt_con_log.columns)

por_95=[]
def capping():
  for i in ele:
    k=np.percentile(list(filter(lambda x:not pd.isna(x), elemnt_con_log[i])),q=95)
    por_95.append(k)
  return por_95

#Listado de percentil 95
pctj_capping=capping()
pctj_capping = tuple(pctj_capping)

header = dict(zip(ele, pctj_capping))


In [ ]:
import ipywidgets as widgets
def graficos(elemento):
  fig, ax = plt.subplots(1,2,figsize=(10, 6))
  sns.kdeplot(ax=ax[0],data=elemnt_con_log,x=elemento,pthresh=header.get(elemento),bins=30,fill=False,kde=True,binwidth=3)
  sns.boxplot(ax=ax[1],data=elemnt_con_log,y=elemento)
  plt.tight_layout()
  plt.show()
widgets.interact(graficos,elemento=ele)



NameError: name 'ele' is not defined